In [87]:
import numpy as np
import pandas as pd
import openpyxl
import random
import gc
import timeit
import sys
from datetime import datetime as dt
import os

In [88]:
filename = "stro_licenses_datasd-san-diego.xlsx"


def get_sheet_names(excel_file):
    workbook = openpyxl.load_workbook(excel_file)
    sheet_names = workbook.sheetnames
    return sheet_names

In [89]:
sheet_name = get_sheet_names(filename)
dfa = pd.read_excel(filename, sheet_name=sheet_name[0], usecols=[0, 1], header=0) # sheet: 'stro_licenses_datasd-san-diego'
dfb = pd.read_excel(filename, sheet_name=sheet_name[2]) # sheet: 'data-apn'

In [90]:
# total rows and columns
row, col = dfa.shape
print('sheet: stro_licenses_datasd-san-diego\nTotal row:\t {} \nTotal colums:\t {}'.format(row, col))

row, col = dfb.shape
print('\nsheet: data-apn\nTotal row:\t {} \nTotal colums:\t {}'.format(row, col))

sheet: stro_licenses_datasd-san-diego
Total row:	 7253 
Total colums:	 2

sheet: data-apn
Total row:	 39620 
Total colums:	 3


In [91]:
dfa.head(3)

,license_id,address
0,STR-01686L,"4855 ALBERSON Ct, SAN DIEGO, CA 92130"
1,STR-01757L,"2028 30th St, San Diego, CA 92104"
2,STR-05313L,"5145 COBAN St, SAN DIEGO, CA 92114"


In [92]:
dfb.head(3)

,permit,apn,parsed_address
0,STR-02005L,4303703300,2070 ILLION ST 92110
1,STR-02053L,4495820800,3611 QUIMBY ST 92106
2,STR-00324L,4236032300,3625 MISSION BLVD 92109


## Data Cleaning:

#### Clean A

In [93]:
# count nan values in columns
dfa.isnull().sum()

license_id    0
address       4
dtype: int64

In [94]:
# Drop rows where specific column values are null / address == null
dfa = dfa.dropna(subset=["address"])

#### Clean B

In [95]:
# Filtering dataframe column elements by different dtypes 
m = dfb['parsed_address'].apply(type) == int
dfb = dfb[~m]
dfb = dfb[~dfb['parsed_address'].str.startswith('#')]
dfb = dfb[~dfb['parsed_address'].str.contains('NONE')]

### Normalize strings address dataframe dfa & dfb

In [96]:
# Convert Pandas Uppercase Column
dfa = dfa.apply(lambda x: x.astype(str).str.lower())
dfb = dfb.apply(lambda x: x.astype(str).str.lower())

In [97]:
# remove extra whitspace, special characters, , #UNIT -> #, #04 -> #4, etc
dfa = (
    dfa.replace({"address": r"\s+"}, {"address": " "}, regex=True) # remove multiple spaces
    .replace({"address": r"\,\s\w+\s\w+(\s,\s|,\s)\w+\s"}, {"address": " "}, regex=True) # ", SAN DIEGO, CA"  --> " "
    .replace({"address": r"\,\s\w+(\s,\s|,\s)\w+\s"}, {"address": " "}, regex=True)
    .replace({"address": r"(\,\s|\s\,)"}, {"address": " "}, regex=True)
    .replace({"address": r"\s(wk)\s"}, {"address": " walk "}, regex=True)
    .replace({"address": r"\s(bl)\s"}, {"address": " blvd "}, regex=True)
    .replace({"address": r"\s(ct)\s"}, {"address": " court "}, regex=True)
    .replace({"address": r"\s(wy)\s"}, {"address": " way "}, regex=True)
    .replace({"address": r"\s(tr)\s"}, {"address": " trails "}, regex=True)
    .replace({"address": r"\s(rw)\s"}, {"address": " row "}, regex=True)
    .replace({"address": r"\s(avenue)\s"}, {"address": " ave "}, regex=True)
    .replace({"address": r"\s(terrance)\s"}, {"address": " ter "}, regex=True)
    .replace({"address": r"\s(av)\s"}, {"address": " ave "}, regex=True)
    .replace({"address": r"\s#unit\s"}, {"address": " #"}, regex=True) #UNIT -> #
    .replace({"address": r"\s#(?=\d\s)"}, {"address": " #0"}, regex=True) # 3243 OCEAN FRONT Wk, #4 SAN DIEGO , CA 92109 -> 3243 OCEAN FRONT Wk, #04 SAN DIEGO , CA 92109
    .replace({"address": r"\s(rd rd)\s"}, {"address": " rd "}, regex=True) # 11942 rancho bernardo rd rd #a 92128 -> 11942 rancho bernardo rd #a 92128
    .replace({"address": r"\s(dr dr)\s"}, {"address": " dr "}, regex=True) # dr dr -> dr
    .replace({"address": r"\s(ave ave)\s"}, {"address": " ave "}, regex=True) # ave ave -> ave
    .replace({"address": r"\s(way way)\s"}, {"address": " way "}, regex=True) # way way -> way
    .replace({"address": r"\s(drive)\s"}, {"address": " dr "}, regex=True) # 11942 rancho bernardo drive #a 92128 -> 11942 rancho bernardo dr #a 92128
    .replace({"address": r"^\d+\s(?=\d{3,}\s)"}, {"address": ""}, regex=True) # 11777 11777 kismet rd 92128 -> 11777 kismet rd 92128
    .replace({"address": r"\."}, {"address": ""}, regex=True) # .
)

# 7185 calabria court ct #b 92122


In [98]:
# remove leading and trailing whitespaces
dfa['address'] = dfa['address'].str.strip()
dfb['parsed_address'] = dfb['parsed_address'].str.strip()

In [99]:
# mkdir storage
path = os.getcwd() + "/storage"


def validate_and_make_folder(folder):
    """validates folders in working directory"""
    try:
        isDir = os.path.isdir(folder) == False
        path = os.getcwd() + "/storage"
        os.mkdir(path)
        print("New folder create:" + path)
    except Exception as e:
        print(e)
        return None


validate_and_make_folder(path)

[Errno 17] File exists: '/home/lakril/Deckard/funzzy-string-match-address/fuzzy_string_matching/storage'


In [100]:
# absolute path
FOLDER_STORAGE = os.path.abspath('storage/')

In [101]:
# export file
dfa.to_csv(os.path.join(FOLDER_STORAGE, r'licenses.csv'), index=False)
dfb.to_csv(os.path.join(FOLDER_STORAGE, r'apn-data.csv'), index=False)

## Comparing and Merge

In [102]:
import fuzzymatcher

In [103]:
# Import the transform data as data frames
licenses = pd.read_csv(FOLDER_STORAGE + "/licenses.csv")
apn_data = pd.read_csv(FOLDER_STORAGE + "/apn-data.csv")

In [104]:
# total rows and columns
row, col = licenses.shape
print('csv: licenses of client\nTotal row:\t {} \nTotal colums:\t {}'.format(row, col))

row, col = apn_data.shape
print('\ncsv: apn-data with suggestion apn by No license\nTotal row:\t {} \nTotal colums:\t {}'.format(row, col))

csv: licenses of client
Total row:	 7249 
Total colums:	 2

csv: apn-data with suggestion apn by No license
Total row:	 36889 
Total colums:	 3


In [105]:
# column level names to join
left_on = ["address"] # licenses data
right_on = ["parsed_address"] # apn data

In [106]:
# Compute Similarity
matched_results = fuzzymatcher.fuzzy_left_join(licenses,
                                               apn_data,
                                               left_on,
                                               right_on,
                                               left_id_col='license_id',
                                               right_id_col='permit')

### Match table to show the similarity scores:

In [107]:
matched_results.sort_values(by=['best_match_score'], ascending=False).head(3)

,best_match_score,__id_left,__id_right,license_id,address,permit,apn,parsed_address
853015,1.250068,str-05286l,str-05286l,str-05286l,2908 lucia jade loop 92139,str-05286l,5913320300,2908 lucia jade loop 92139
1201702,1.158014,str-07854l,str-07854l,str-07854l,10510 corte jardin del mar 92130,str-07854l,3076400800,10510 corte jardin del mar 92130
74258,1.136083,str-02692l,str-02692l,str-02692l,12811 calle de las rosas 92129,str-02692l,3154213600,12811 calle de las rosas 92129


In [108]:
matched_results.sort_values(by=['best_match_score'], ascending=True).head(3)

,best_match_score,__id_left,__id_right,license_id,address,permit,apn,parsed_address
709222,-0.675630,str-06705l,str-01686l,str-06705l,6140 galante place 92130,str-01686l,3043700600,4855 alberson court 92130
1220240,-0.585137,str-07383l,str-01128l,str-07383l,8430 via sonoma #no.50 92037,str-01128l,3467610400,8425 n la jolla scenic dr 92037
934375,-0.534950,str-06318l,str-00839l,str-06318l,4120 porte de merano #82 92122,str-00839l,3011220200,2583 via merano 92014


In [109]:
result = matched_results.query("best_match_score <= .80").sort_values(
    by=['best_match_score'], ascending=False)

In [110]:
# total rows and columns
row, col = result.shape
print('result: fuzzy matches\nTotal row:\t {} \nTotal colums:\t {}'.format(row, col))

# Column Names, missing values and memory usage
name_col = result.columns.values.tolist()
print('\nColumn names: {} \n '.format(name_col))

result: fuzzy matches
Total row:	 32501 
Total colums:	 8

Column names: ['best_match_score', '__id_left', '__id_right', 'license_id', 'address', 'permit', 'apn', 'parsed_address'] 
 


In [111]:
# match
def match(row):
    match = 0
    if (row['__id_left'] == row['__id_right']) and (row['address'] == row['parsed_address']):
        match = True
    else:
        total = False
    return match

In [112]:
result["full_match"] = result.apply(match, axis=1)

In [113]:
# filter full match
full_match_true = result[result['full_match']== True]
full_match_true

# total rows and columns
row, col = full_match_true.shape
print('Full match license No & address:\nTotal row:\t {} \nTotal colums:\t {}'.format(row, col))

Full match license No & address:
Total row:	 3251 
Total colums:	 9


In [114]:
# export file
result.to_csv(os.path.join(FOLDER_STORAGE, r'fuzzymatcher.csv'), index=False)